In [12]:
#import general dependencies
import pandas as pd
import plotly.express as px
import hvplot.pandas
import matplotlib.pyplot as plt

In [13]:
#read in the data
#may need to request the data from a postgres database
df = pd.read_csv('../../Movie_Rec_Data/movies_merged.csv')

In [14]:
#check data shape, dtypes, etc.
df.shape

(45413, 36)

In [15]:
df.dtypes

index                      int64
adult                       bool
budget                     int64
id                         int64
imdb_id                    int64
original_language         object
popularity               float64
poster_path               object
release_date              object
revenue                    int64
runtime                  float64
status                    object
title                     object
video                       bool
vote_average             float64
vote_count                 int64
genre_action             float64
genre_adventure          float64
genre_animation          float64
genre_comedy             float64
genre_crime              float64
genre_documentary        float64
genre_drama              float64
genre_family             float64
genre_fantasy            float64
genre_foreign            float64
genre_history            float64
genre_horror             float64
genre_music              float64
genre_mystery            float64
genre_roma

In [16]:
df.head()

,index,adult,budget,id,imdb_id,original_language,popularity,poster_path,release_date,revenue,...,genre_history,genre_horror,genre_music,genre_mystery,genre_romance,genre_science_fiction,genre_tv_movie,genre_thriller,genre_war,genre_western
0,4,False,0,11862,113041,en,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8,False,35000000,9091,114576,en,5.231580,/eoWvKD60lT95Ss1MYNgVExpo5iU.jpg,1995-12-22,64350171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,9,False,58000000,710,113189,en,14.686036,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,1995-11-16,352194034,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,10,False,62000000,9087,112346,en,6.318445,/lymPNGLZgPHuqM29rKMGV46ANij.jpg,1995-11-17,107879496,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,11,False,0,12110,112896,en,5.430331,/xve4cgfYItnOhtzLYoTwTVy5FGr.jpg,1995-12-22,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
#drop unhelpful columns without too much exploration
del df['index']
del df['poster_path']

In [27]:
#finding all values in the video column that are true. presumably these are tv shows
tv_shows = df[df.video]

In [40]:
# turns out, most of these ranged from documentaries to video game cinematics. 
tv_shows[['video', 'title']]

,video,title
3542,True,Love Hina Spring Special - I Wish Your Dream
4627,True,Yes: 9012 live
4878,True,Tank on the Moon
4918,True,Step Up Love Story
5117,True,911 in Plane Site
...,...,...
44961,True,Devo: The Complete Truth About De-Evolution
45071,True,The Godfather Trilogy: 1972-1990
45261,True,Salad Fingers
45280,True,The End of a Vacation


In [41]:
tv_shows = tv_shows.index

In [42]:
df.drop(tv_shows, axis=0)

,adult,budget,id,imdb_id,original_language,popularity,release_date,revenue,runtime,status,...,genre_history,genre_horror,genre_music,genre_mystery,genre_romance,genre_science_fiction,genre_tv_movie,genre_thriller,genre_war,genre_western
0,False,0,11862,113041,en,8.387519,1995-02-10,76578911,106.0,Released,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,35000000,9091,114576,en,5.231580,1995-12-22,64350171,106.0,Released,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,False,58000000,710,113189,en,14.686036,1995-11-16,352194034,130.0,Released,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,False,62000000,9087,112346,en,6.318445,1995-11-17,107879496,106.0,Released,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,False,0,12110,112896,en,5.430331,1995-12-22,0,88.0,Released,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45408,False,0,106807,135571,fr,0.225432,1900-01-01,0,2.0,Released,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45409,False,0,276895,3054038,en,0.011025,1981-01-01,0,58.0,Released,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45410,False,0,404604,5690142,hi,1.559596,2017-07-07,0,146.0,Released,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
45411,False,0,30840,102797,en,5.683753,1991-05-13,0,104.0,Released,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.head()

In [ ]:
#scale the data
from sklearn.preprocessing import StandardScaler

df_scaled = StandardScaler.fit_transform(df)

In [ ]:
#using NearestNeighbors to help optimize our dbscans model
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=)
nbrs = neigh.fit(df)
distances, indices = nbrs.kneighbors(df[[]])

In [ ]:
# Plotting K-distance Graph to find our epsilon value
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.figure(figsize=(20,10))
plt.plot(distances)
plt.title('K-distance Graph',fontsize=20)
plt.xlabel('Data Points sorted by distance',fontsize=14)
plt.ylabel('Epsilon',fontsize=14)
plt.show()

In [ ]:
#Potentially use sklearn's dbscan model to find outliers before clustering with kmeans
from sklearn.cluster import DBSCAN

model = DBSCAN(esp=0.3, min_samples=15)

In [ ]:
# outliers dataframe
outliers = df[model.labels_ == -1]
print(outliers)

In [ ]:
#import our dependencies
from sklearn.cluster import KMeans
# from sklearn import metrics
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer(stop_words='english')
# X = vectorizer.fit_transform(df_scaled)

In [ ]:
#determine our clusters using an elbow plot
inertia = []
clusters = list(range(1,21))

# Calculate the inertia for the range of K values
for i in clusters:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit()
    inertia.append(model.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"clusters": clusters, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="clusters", y="inertia", title="Elbow Curve", xticks=clusters)

In [ ]:
#build function for running a Kmeans model

def get_Kmeans_results(): 
    #instantiate the classifier
    model = Kmeans(n_clusters = 5, init='k-means++', max_iter=250, n_init=15)

    #fit our scaled data
    model = model.fit(df)

    #make predictions
    predictions = model.predict(df)

    #create a column in our dataframe for our classifications
    df["Group"] = model.labels_

    return df

In [ ]:
#analyze labels, see if we need to change clusters

In [ ]:
#plot our clusters
import plotly.express as px
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    df,
    x='',
    y='',
    z='',
    color='',
    symbol='',
    hover_name='',
    hover_data=[''],
    width=800
)
fig.show()